In [1]:
%load_ext autoreload
%autoreload 2

import os,sys
parent_dir = os.path.abspath('..')
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

import util as yu
from util import *

folder='A20_allcases'
yu.setpath_fig(f'fig/{folder}/fig_ignore/')
yu.setpath_pkl(f'pkl/{folder}/fig_ignore/')

projs=['P0', 'Px', 'Py', 'Pz']
inserts=['tt', 'tx', 'ty', 'tz', 'xx', 'xy', 'xz', 'yy', 'yz', 'zz']
ens='b'

In [2]:
basepath='/p/project1/ngff/li47/code/projectData/05_moments/cB211.072.64/data_merge/'
path=f'{basepath}disc_2pt.h5'
with h5py.File(path) as f:
    moms_2pt=yu.moms2list(f['moms'])
    c2pt=yu.jackknife(np.real(f['data/N_N'][:,:,:]))
    
meff=yu.jackmap(yu.c2pt2meff,c2pt[:,:,moms_2pt.index([0,0,0])])
fitss_2pt=yu.doFit_meff_nst(meff,[range(8,24+1),range(1,10+1),range(1,4+1)],[0.4,0.5,2,0.8,1],downSampling=2,label='meff')
fig,axd=yu.makePlot_2pt_SimoneStyle(meff,fitss_2pt,xunit=yu.ens2a[ens],yunit=yu.ens2aInv[ens]/1000,mN_exp=0.938)
yu.finalizePlot('meff')

pars_jk_meff2st,props_jk=yu.jackMA(fitss_2pt[1])
mN_2stMA=pars_jk_meff2st[:,0]

# results from different momenta of nucleon

In [12]:
basepath='/p/project1/ngff/li47/code/scratch/run/05_moments_run5/cB211.072.64/data_merge/'
files=os.listdir(basepath)

momjtf2c3pt_tt={}
momjtf2c3pt_ti={}
momstrs=[]
for file in files:
    momstr=file[5:-3]
    momstrs.append(momstr)
    with h5py.File(f'{basepath}{file}') as f:    
        for jtf in f['data'].keys():
            j,tf=jtf.split('_')
            if j.startswith('jg') and j not in ['jg;stout10','jg;stout20']:
                continue
            tf=int(tf)
            t=f['data'][jtf][:]
            
            momjtf2c3pt_tt[(momstr,j,tf)]=yu.jackknife(t[:,:,0,projs.index('P0'),inserts.index('tt')])
            momjtf2c3pt_ti[(momstr,j,tf)]=yu.jackknife(t[:,:,0,projs.index('P0'),inserts.index('tz')])

In [ ]:
tfs=range(2,20)
js=['j+;disc','js;disc','jc;disc','jg;stout10','jg;stout20']
for ij,j in enumerate(js):
    momstr2tf2ratio={}; momstr2fits={}
    for ims,momstr in enumerate(momstrs):
        print(f'{ij}/{len(js)}; {ims}/{len(momstrs)}',end='                 \r')
        
        mom=[int(ele) for ele in momstr.split(',')]
        momN=mom[:3]
        if mom[2]==0:
            continue
        factor=1/(1j*2*np.pi/yu.ens2NL[ens]*mom[2])
        tf2ratio={}
        for tf in tfs:
            c3pt=momjtf2c3pt_ti[(momstr,j,tf)]
            tf2ratio[tf]=np.real(c3pt/c2pt[:,tf:tf+1,moms_2pt.index(momN)]*factor)
        tf2ratio=yu.symmetrizeRatio(tf2ratio)
        momstr2tf2ratio[momstr]=tf2ratio
        momstr2fits[momstr]=yu.doFit_3pt_1st(tf2ratio,tfs,[2,3,4,5,6],downSampling=[2,2],symmetrizeQ=True)
        
        
    momstrs_plt=list(momstr2tf2ratio.keys())
    nvecs=[[int(ele) for ele in momstr.split(',')[:3]] for momstr in momstrs_plt]
    n2s=[yu.nvec2n2(nvec) for nvec in nvecs]
    
    figs=[]
    
    list_tf2ratio_fits=[(momstr2tf2ratio[momstr],momstr2fits[momstr]) for momstr in momstrs_plt]
    fig,axs,pars_jk_quotes=yu.makePlot_3pt_ChristosStyle(list_tf2ratio_fits,xunit=yu.ens2a[ens],tcmin_rainbow=2,ylabels=nvecs)
    fig.suptitle(j,fontsize=32)
    yu.finalizePlot(closeQ=True)
    figs.append(fig)
    
    xs=n2s
    y_jk=np.transpose([pars_jk[:,0] for pars_jk in pars_jk_quotes])
    fig,axs=yu.makePlot_simpleComparison(xs,y_jk)
    ax=axs[0,0]
    ax.set_xlabel('msq')
    # pars_jk,_,_=yu.doFit_const(y_jk,corrQ=False)
    # mean,err=yu.jackme(pars_jk)
    # plt_x=ax.get_xlim(); plt_y=mean[0]; plt_yerr=err[0]
    # ax.fill_between(plt_x,plt_y-plt_yerr,plt_y+plt_yerr,color='red',alpha=0.2,label=yu.un2str(plt_y,plt_yerr))  
    # ax.legend(fontsize=16)
    
    yu.finalizePlot(closeQ=True)
    figs.append(fig)
    
    yu.makePDF(f'unequal_{j}',figs)

# ratio fit using different effecitve-mass-fit input
'model average (MA)' vs 'most probable (MP)' vs 'earlist stable (ES)'

In [9]:
basepath='/p/project1/ngff/li47/code/projectData/05_moments/cB211.072.64/data_merge/'

path=f'{basepath}/conn_0,0,0,0,0,0.h5'
with h5py.File(path) as f:
    moms=yu.moms2list(f['moms'])
    imom=moms.index([0,0,0,0,0,0])
    
    tf2c3pt_jp={}; tf2c3pt_jm={}
    for jtf in f['data'].keys():
        j,tf=jtf.split('_')
        tf=int(tf)
        t=f[f'data/{jtf}'][:]
        t=t[:,:,0,projs.index('P0'),inserts.index('tt')]
        t=yu.jackknife(t)

        if j in ['j+;conn']:
            tf2c3pt_jp[tf]=t
        elif j in ['j-;conn']:
            tf2c3pt_jm[tf]=t
        
path=f'{basepath}/conn_2pt.h5'
with h5py.File(path) as f:
    moms=yu.moms2list(f['moms'])
    imom=moms.index([0,0,0])
    
    tf2c2pt={}
    for tf in f['data'].keys():
        t=f[f'data/{tf}'][:]
        t=yu.jackknife(np.real(t[:,:,imom]))
        tf2c2pt[int(tf)]=t

In [10]:
pars_jk_meff2st,props_jk=yu.jackMA(fitss_2pt[1])
mN_jk=pars_jk_meff2st[:,0]
factor_equal=1/(-3*mN_jk/4)
tf2ratio={}
for tf in tf2c3pt_jp.keys():
    tf2ratio[tf]=np.real(tf2c3pt_jp[tf]/tf2c2pt[tf][:,tf:tf+1]*factor_equal[:,None])
    
pars_jk_meff2st_MA=pars_jk_meff2st
tf2ratio_MA=tf2ratio

_,props_jk=yu.jackMA(fitss_2pt[1])
ind_mpf=np.argmax(np.mean(props_jk,axis=0))
pars_jk_meff2st=fitss_2pt[1][ind_mpf][1]
mN_jk=pars_jk_meff2st[:,0]
factor_equal=1/(-3*mN_jk/4)
tf2ratio={}
for tf in tf2c3pt_jp.keys():
    tf2ratio[tf]=np.real(tf2c3pt_jp[tf]/tf2c2pt[tf][:,tf:tf+1]*factor_equal[:,None])
    
pars_jk_meff2st_MP=pars_jk_meff2st
tf2ratio_MP=tf2ratio

_,props_jk=yu.jackMA(fitss_2pt[1])
ind_mpf=np.argmax(np.mean(props_jk,axis=0))
pars_jk_meff2st=fitss_2pt[1][ind_mpf-1][1]
mN_jk=pars_jk_meff2st[:,0]
factor_equal=1/(-3*mN_jk/4)
tf2ratio={}
for tf in tf2c3pt_jp.keys():
    tf2ratio[tf]=np.real(tf2c3pt_jp[tf]/tf2c2pt[tf][:,tf:tf+1]*factor_equal[:,None])
    
pars_jk_meff2st_ES=pars_jk_meff2st
tf2ratio_ES=tf2ratio

tfmins=[8,10,12,14,16]; tcmins=[2,3,4]
# fits_MA=yu.doFit_3ptSym_2st2step(tf2ratio_MA,tfmins,tcmins,pars0,pars_jk_meff2st_MA,downSampling=[1,2],symmetrizeQ=True)
# fits_MP=yu.doFit_3ptSym_2st2step(tf2ratio_MP,tfmins,tcmins,pars0,pars_jk_meff2st_MP,downSampling=[1,2],symmetrizeQ=True)
# fits_ES=yu.doFit_3ptSym_2st2step(tf2ratio_ES,tfmins,tcmins,pars0,pars_jk_meff2st_ES,downSampling=[1,2],symmetrizeQ=True)

fits_MA=yu.doFit_3ptSym_2st2step(tf2ratio_MA,tfmins,tcmins,pars_jk_meff2st_MA,downSampling=[1,1],symmetrizeQ=False)
fits_MP=yu.doFit_3ptSym_2st2step(tf2ratio_MP,tfmins,tcmins,pars_jk_meff2st_MP,downSampling=[1,1],symmetrizeQ=False)
fits_ES=yu.doFit_3ptSym_2st2step(tf2ratio_ES,tfmins,tcmins,pars_jk_meff2st_ES,downSampling=[1,1],symmetrizeQ=False)

In [11]:
list_tf2ratio_fits=[
    (tf2ratio_MA,fits_MA,pars_jk_meff2st_MA),
    (tf2ratio_MP,fits_MP,pars_jk_meff2st_MP),
    (tf2ratio_ES,fits_ES,pars_jk_meff2st_ES),
]
ylabels=[r'$\langle x \rangle_{u+d}^{\rm conn, B}$ (MA)',
         r'$\langle x \rangle_{u+d}^{\rm conn, B}$ (MP)',
         r'$\langle x \rangle_{u+d}^{\rm conn, B}$ (ES)',
]

yu.makePlot_3pt_ChristosStyle(list_tf2ratio_fits,xunit=yu.ens2a[ens],tcmin_rainbow=2,ylabels=ylabels)
yu.finalizePlot(f'MAvsMPvsES')

# correlated ratio vs uncorrelated ratio

In [3]:
basepath='/p/project1/ngff/li47/code/projectData/05_moments/cB211.072.64/data_merge/'

path=f'{basepath}/conn_0,0,0,0,0,0.h5'
with h5py.File(path) as f:
    moms=yu.moms2list(f['moms'])
    imom=moms.index([0,0,0,0,0,0])
    
    tf2c3pt_jp={}; tf2c3pt_jm={}
    for jtf in f['data'].keys():
        j,tf=jtf.split('_')
        tf=int(tf)
        t=f[f'data/{jtf}'][:]
        t=t[:,:,0,projs.index('P0'),inserts.index('tt')]
        t=yu.jackknife(t)

        if j in ['j+;conn']:
            tf2c3pt_jp[tf]=t
        elif j in ['j-;conn']:
            tf2c3pt_jm[tf]=t
        
path=f'{basepath}/conn_2pt.h5'
with h5py.File(path) as f:
    moms=yu.moms2list(f['moms'])
    imom=moms.index([0,0,0])
    
    tf2c2pt={}
    for tf in f['data'].keys():
        t=f[f'data/{tf}'][:]
        t=yu.jackknife(np.real(t[:,:,imom]))
        tf2c2pt[int(tf)]=t

In [4]:
pars_jk_meff2st,props_jk=yu.jackMA(fitss_2pt[1])
mN_jk=pars_jk_meff2st[:,0]
factor_equal=1/(-3*mN_jk/4)

tf2ratio_jp={}
for tf in tf2c3pt_jp.keys():
    tf2ratio_jp[tf]=np.real(tf2c3pt_jp[tf]/tf2c2pt[tf][:,tf:tf+1]*factor_equal[:,None])
    
tf2ratio_jm={}
for tf in tf2c3pt_jp.keys():
    tf2ratio_jm[tf]=np.real(tf2c3pt_jm[tf]/tf2c2pt[tf][:,tf:tf+1]*factor_equal[:,None])
    
tfmins=[8,10,12,14,16]; tcmins=[2,3,4]
fits_jp=yu.doFit_3ptSym_2st2step(tf2ratio_jp,tfmins,tcmins,pars_jk_meff2st,downSampling=[2,2],symmetrizeQ=True)
fits_jm=yu.doFit_3ptSym_2st2step(tf2ratio_jm,tfmins,tcmins,pars_jk_meff2st,downSampling=[2,2],symmetrizeQ=True)

tf2ratio_jp_uncorr={}
for tf in tf2c3pt_jp.keys():
    tf2ratio_jp_uncorr[tf]=np.real(tf2c3pt_jp[tf]/c2pt[:,tf:tf+1,moms_2pt.index([0,0,0])]*factor_equal[:,None])
    
tf2ratio_jm_uncorr={}
for tf in tf2c3pt_jp.keys():
    tf2ratio_jm_uncorr[tf]=np.real(tf2c3pt_jm[tf]/c2pt[:,tf:tf+1,moms_2pt.index([0,0,0])]*factor_equal[:,None])
    
tfmins=[8,10,12,14,16]; tcmins=[2,3,4]
fits_jp_uncorr=yu.doFit_3ptSym_2st2step(tf2ratio_jp_uncorr,tfmins,tcmins,pars_jk_meff2st,downSampling=[2,2],symmetrizeQ=True)
fits_jm_uncorr=yu.doFit_3ptSym_2st2step(tf2ratio_jm_uncorr,tfmins,tcmins,pars_jk_meff2st,downSampling=[2,2],symmetrizeQ=True)

In [5]:
tfmins=[8,10,12,14,16]; tcmins=[2,3,4]
fits=yu.doFit_3ptSym_2st2step(tf2ratio_jp,tfmins,tcmins,pars_jk_meff2st*0+np.mean(pars_jk_meff2st,axis=0)[None,:],downSampling=[1,1],symmetrizeQ=False)
pars_jk,props_jk=yu.jackMA(fits)
mean,err=yu.jackme(pars_jk)
print(yu.un2str(mean[0],err[0]))
for fit in fits:
    (tfmin,tcmin),pars_jk,chi2_jk,Ndof=fit
    mean,err=yu.jackme(pars_jk)
    print(tfmin,tcmin,yu.un2str(mean[0],err[0]))

0.3818(92)
8 2 0.3808(67)
8 3 0.3578(82)
8 4 0.370(10)
10 2 0.3802(72)
10 3 0.3614(84)
10 4 0.371(10)
12 2 0.3888(78)
12 3 0.3699(88)
12 4 0.382(10)
14 2 0.3920(87)
14 3 0.3790(95)
14 4 0.385(11)
16 2 0.391(12)
16 3 0.382(12)
16 4 0.382(14)


In [8]:
list_tf2ratio_fits=[
    (tf2ratio_jp,fits_jp,pars_jk_meff2st),
    (tf2ratio_jp_uncorr,fits_jp_uncorr,pars_jk_meff2st),
    (tf2ratio_jm,fits_jm,pars_jk_meff2st),
    (tf2ratio_jm_uncorr,fits_jm_uncorr,pars_jk_meff2st),
]
ylabels=[r'$\langle x \rangle_{u+d}^{\rm B, conn}$ (corr)',
         r'$\langle x \rangle_{u+d}^{\rm B, conn}$ (unco)',
         r'$\langle x \rangle_{u-d}^{\rm B, conn}$ (corr)',
         r'$\langle x \rangle_{u-d}^{\rm B, conn}$ (unco)'
]

yu.makePlot_3pt_ChristosStyle(list_tf2ratio_fits,xunit=yu.ens2a[ens],tcmin_rainbow=2,ylabels=ylabels)
yu.finalizePlot(f'corrRatio_vs_uncorrRatio')
None